# College Football Simulation with new pairing system
### By Rodrigo Vargas and Daniel Yedidia
Project for PIC-16B Python with Applications II, UCLA Spring 2023 <br>
Github: https://github.com/dyeds/PIC-16B-Project

Structure: (Tentative)

1. Information & Credits: CollegeFootballData.com, https://www.reddit.com/r/CFB/comments/qq553i/what_would_college_football_look_like_under_a/ Post by u/dethwing_.
2. Explanation of the project and how to use the notebook
3. Data Acquisition and Preprocessing. Working with CFBD and BingMaps API's and storing locally on Database
4. Model creation using Tensorflow. Creating various models and motives on why to use Betting Lines.
5. Explanation and Implementation of Minimum Weight Matching Algorithm using Networkx for pairings and using distances.
6. SQL Database with Simulated Games and other table creating Functions.
7. Visualizations of Simulation using Plotly.
8. Biases and Future Improvements for the project. 

## Information & Credits

## Project Explanations and Instructions

## Data Acquisition and Preprocessing

## Predictive Model using Tensorflow

## Pairing System using Minimum Weight Matching Algorithm

## Simulated Results

## Graphical Visualizations of Results

## Further Improvements